In [62]:
import sys
print (sys.version)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.basemap import Basemap
from itertools import chain
import math
%matplotlib inline

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]



# Project WP1-2
### code 1: Clean journey and remove duplicates
<span style='font-family:"Times New Roman"'> <span styel=''> 
This code is used to extract the "PortList.csv" information from "fleet_trade_journeys.csv"; 
Duplicate terminals are defined as terminals with the same names but in different countries. There are 38 duplicated entries in the dataset "PortList.xlsx" from Gang, which needs to be improved;
The original dataset is "fleet_trade_journeys.csv";
### Basic steps: 
    a) Load data;
    b) Check duplicate entries in "portList.xlsx";
    c) Extract all duplicate terminals/and its inforamtion from original dataset.
    d) Find the number of calls for new cleaned dataset
    e) Save the cleaned dataset

***
### Step a):  load data in both "fleet_trade_journeys.csv" and "PortList.xlsx", and check the statistic information of data.
***

In [63]:
df_journey=pd.read_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/fleet_trade_journeys.csv")
df_ports=pd.ExcelFile("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/PortList.xlsx")
df_ports=df_ports.parse(header=0,index_col=0)

In [64]:
df_journey.head()

,journey_id,ship_id,ais_ship_id,journey_laden,journey_start_stoppage_id,journey_start_timestamp,journey_start_draft,journey_start_latitude,journey_start_longitude,journey_start_location_type,...,journey_end_location_name,journey_end_location_country,journey_end_location_unlocode,journey_distance_nm,journey_underway_calc_avg_speed,journey_processed,journey_current,journey_estimated_cargo_tons,journey_underway_distance,journey_underway_time_hours
0,286559091,9301421,97,1,886369008,2013-12-30 05:34:03,9.0,51.45277,140.92767,Port,...,Yeosu,South Korea,KRYOS,1247.352457,13.959216,2019-03-28 06:53:38,0,98235,1195.141506,85.616667
1,286559092,9301421,97,0,886369009,2014-01-07 13:30:04,14.2,34.71473,127.81125,Anchorage,...,De Kastri,Russia,RUDKA,1251.511628,13.074002,2019-03-28 06:53:38,0,0,1233.310535,94.333056
2,286559093,9301421,97,1,886369012,2014-01-19 01:02:19,9.0,51.48012,140.92478,Anchorage,...,Ulsan,South Korea,KRUSN,1166.283499,14.453006,2019-03-28 06:53:38,0,98235,1132.023627,78.324444
3,286559094,9301421,97,0,886369014,2014-01-25 05:06:44,14.2,35.44117,129.39453,Port,...,De Kastri,Russia,RUDKA,1339.352542,13.024184,2019-03-28 06:53:38,0,0,1197.562825,91.949167
4,286559095,9301421,97,1,886369018,2014-02-05 12:23:14,9.0,51.48157,140.92332,Port,...,Gwangyang,South Korea,KRKAN,1299.165114,13.671880,2019-03-28 06:53:38,0,98235,1288.684831,94.258056


In [65]:
df_journey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501331 entries, 0 to 501330
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   journey_id                       501331 non-null  int64  
 1   ship_id                          501331 non-null  int64  
 2   ais_ship_id                      501331 non-null  int64  
 3   journey_laden                    501331 non-null  int64  
 4   journey_start_stoppage_id        501331 non-null  int64  
 5   journey_start_timestamp          501331 non-null  object 
 6   journey_start_draft              501331 non-null  float64
 7   journey_start_latitude           501331 non-null  float64
 8   journey_start_longitude          501331 non-null  float64
 9   journey_start_location_type      501292 non-null  object 
 10  journey_start_location_name      501292 non-null  object 
 11  journey_start_location_country   500725 non-null  object 
 12  jo

In [66]:
df_journey.describe().T

,count,mean,std,min,25%,50%,75%,max
journey_id,501331.0,2.875341e+08,3.485398e+06,2.734272e+08,2.841194e+08,2.865120e+08,2.913336e+08,2.924330e+08
ship_id,501331.0,9.430823e+06,2.884781e+05,6.329044e+06,9.274317e+06,9.351452e+06,9.467732e+06,1.010406e+07
ais_ship_id,501331.0,1.366647e+05,2.280218e+05,9.700000e+01,1.541800e+04,4.786100e+04,1.018680e+05,1.030385e+06
journey_laden,501331.0,5.441255e-01,4.980496e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
journey_start_stoppage_id,501331.0,8.875433e+08,7.886687e+06,8.530603e+08,8.797651e+08,8.862683e+08,8.956614e+08,8.988223e+08
journey_start_draft,501331.0,9.404650e+00,2.022961e+00,0.000000e+00,8.000000e+00,8.900000e+00,1.080000e+01,2.550000e+01
journey_start_latitude,501331.0,2.582539e+01,2.236658e+01,-5.327544e+01,1.265715e+01,2.974437e+01,4.060922e+01,7.655333e+01
journey_start_longitude,501331.0,2.232075e+01,7.662933e+01,-1.717647e+02,-4.632785e+01,2.350883e+01,1.035428e+02,1.793667e+02
journey_end_stoppage_id,499683.0,8.875967e+08,7.901345e+06,8.530603e+08,8.797677e+08,8.862969e+08,8.956848e+08,8.988223e+08
journey_end_draft,499683.0,9.417424e+00,1.996592e+00,0.000000e+00,8.000000e+00,8.900000e+00,1.080000e+01,2.550000e+01


In [67]:
df_ports.head()

,Country,Terminal,Port Group,Region,Lat,Lon,num of calls
index,,,,,,,
6,Argentina,Rio Cullen,Rio Cullen,East Sou.Ame,-52.804410,-68.208910,56
13,Argentina,Punta Loyola,Rio Cullen,East Sou.Ame,-51.606985,-69.019225,14
17,New Zealand,Dunedin,Bluff,Oceania,-45.875160,170.519330,26
18,Argentina,Comodoro Rivadavia,Caleta Cordova,East Sou.Ame,-45.860180,-67.389045,40
19,Argentina,Caleta Cordova,Caleta Cordova,East Sou.Ame,-45.773050,-67.320400,447


***
### Step b):  Check duplicate entries in "portList.xlsx".
***

In [68]:
# obtain duplicate terminals based on index
# then concatenate them into a dataframe
pd_dupPorts=pd.concat(g for _, g in df_ports.groupby("index") if len(g) > 1)
print ("The duplicate terminals extracted from Gang's file")
print ("-------------------------------------------------------------------------------------------------------------")
print (pd_dupPorts.info())
print ("-------------------------------------------------------------------------------------------------------------")
print ('\n')

# extract duplicate terminals and their indices
ix_dup=pd_dupPorts.index
ix_dup=ix_dup.values
ix_dup=list(set(ix_dup))
terminal_dup=pd_dupPorts.Terminal
terminal_dup=terminal_dup.values
terminal_dup=list(set(terminal_dup))

print ("The duplicated index of terminals:\n",ix_dup)
print ("---------------------------------------------------------------------------------------------------------------------")
print ("The corresponding duplicated terminals:\n",terminal_dup)
print ("---------------------------------------------------------------------------------------------------------------------")
print("The number of duplicated terminals based on this file:")
print (len(ix_dup))
print (len(terminal_dup))
print ("---------------------------------------------------------------------------------------------------------------------")
# double check for these duplicated terminals in the original file "fleet_trade_journeys.csv".

df_journeyLoc=df_journey[["journey_start_location_type","journey_start_location_country",
            "journey_start_location_name","journey_start_latitude",  "journey_start_longitude", 
            "journey_end_location_type","journey_end_location_country","journey_end_location_name",
             "journey_end_latitude",  "journey_end_longitude"]]
# take the duplicated case "Valdez Marine Terminal" for an example, we check if it is a read duplicated one in original file.
df_ex=df_journeyLoc[df_journeyLoc.journey_start_location_name=="Valdez Marine Terminal"]
df_ex1=df_journeyLoc[df_journeyLoc.journey_end_location_name=="Valdez Marine Terminal"]
print ("---------------------------------------------------------------------------------------------------------------------")
print ("We found that the overall number of journey starting from 'Valdez Marine Terminal' is {},\n all belong to the country: \n {}".format(len(df_ex),df_ex["journey_start_location_country"].value_counts()))
print ("---------------------------------------------------------------------------------------------------------------------")
print ("The same situation with the other terminals.")
print ("---------------------------------------------------------------------------------------------------------------------")
print ("Therefore, the so-called duplicated terminals are not correct.")
print ("A entire cleanning of the journey dataset is needed!")

The duplicate terminals extracted from Gang's file
-------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 4743 to 4780
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       76 non-null     object 
 1   Terminal      76 non-null     object 
 2   Port Group    76 non-null     object 
 3   Region        76 non-null     object 
 4   Lat           76 non-null     float64
 5   Lon           76 non-null     float64
 6   num of calls  76 non-null     int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 4.8+ KB
None
-------------------------------------------------------------------------------------------------------------


The duplicated index of terminals:
 [4743, 4744, 4745, 4746, 4747, 4748, 4749, 4750, 4751, 4752, 4753, 4754, 4755, 4756, 4757, 4758, 4759, 4760, 4761, 4762, 4763, 476

### Step c): Extract all duplicate terminals from original dataset. 
Clean the data and fill all the missing data.

In [69]:
def check_null(df,column="Country"):
    """
    check if there are Null data in the given dataframe
    df: dataframe
    column: string, the name of checked column
    
    return: a dataframe with all the null data
    """
    if column is None: 
        return 
    
    df_null=df[df[column].isnull()]
    return df_null

def check_missData(df):
    """
    check the missing data of dataFrame.
    return: a dataFrame describing missing data percentage
    sum(): count the number of missing data in each column
    isnull(): return a dataFrame with True and False   
    """
    miss_tot=df.isnull().sum().sort_values(ascending=False)
    counts_all=df.isnull().count() # count all the elements, including the missed elements
    miss_per=((df.isnull().sum())*100/counts_all).sort_values(ascending=False)
    miss_all=pd.concat([miss_tot,miss_per],axis=1,keys=['TotalNum','TotalPerc(\%)'])
    return miss_all


def fill_null(df,column="Country",value="unknown"):
    """
    Fill the NaN in given column with given value
    In case for modification, it will be updated as required.
    
    df: dataframe
    column: string, the column with Null
    value: string, the value to be filled
    return: a dataframe with filled value
    """
    df[column].fillna(value,inplace=True)
    return df

def check_dup(df,check_term=["Mongstad Refinery","Slovag"]):
    """
    # check the original journey file if there are duplicate terminal names in different countries
    # only suitable for dataframe with columns: journey_start_location_name,journey_end_location_name,
    # journey_start_location_country and journey_end_location_country.
    # checking criterion: whether or not the length of grouped dataframe with given terminals
    # is equal to the first value of list in counted country.
    # if so--> no duplicate terminals, otherwise, with duplicate terminals.
    # df: dataframe
    # check_term: checked terminals
    # return: a list including all the duplicate terminals in given dataframe.

    """
    if check_term is None:
        return
    
    dups=[]
    for terminal in check_term:
        df_start_term=df[df.journey_start_location_name==terminal]
        df_end_term=df[df.journey_end_location_name==terminal]
        
        if (terminal in df.journey_start_location_name.values):
            len_start=len(df_start_term)
            counts_start=df_start_term["journey_start_location_country"].value_counts()
            counts_start=counts_start.values
            
            if counts_start !=[]:
                if (len_start!=counts_start[0]):
                    dups.append(terminal)
                
        if (terminal in df.journey_end_location_name.values):
            len_end=len(df_end_term)
            counts_end=df_end_term["journey_end_location_country"].value_counts()
            counts_end=counts_end.values
            
            if counts_end !=[]:
                if (len_end!=counts_end[0]):
                    dups.append(terminal)
                    
        dups=list(set(dups))
        
    return dups

def df_reduce(df_origin,duplicate_terms=['Cartagena', 'Vancouver']):
    """
    obtain the shrinked datafrome with given terminals.
    dataFrame format should be the same as original journey file
    
    df_origin: original dataframe
    duplicate_terms: duplicate terminals, list
    return: a dataframe with rows including only given terminals. 
    """
    ix=[]
    
    if duplicate_terms is None:
        return
    
    cols=["journey_start_location_name","journey_end_location_name"]
    for terminal in duplicate_terms:
        
        if terminal in df_origin[cols[0]].values :
            temp=df_origin[df_origin[cols[0]]==terminal].index.tolist()
            ix.extend(temp)
        if terminal in df_origin[cols[1]].values:
            temp=df_origin[df_origin[cols[1]]==terminal].index.tolist()
            ix.extend(temp)
            
    ix=list(set(ix))
    df_new=df_origin.loc[ix]
    
    return df_new

def updateName_dup(df,duplicates=['Cartagena', 'Vancouver']):
    """
    update the name for duplicates teminals
    Note that this function will modify the data in given dataset.
    
    df:dataframe
    duplicates: list, names of duplicates terminals
    return: a dataframe with no duplicate entries anymore
    """
 
    locations=["journey_start_location_name","journey_end_location_name"]
    countries=["journey_start_location_country","journey_end_location_country"]
    
    if duplicates is None:
        return
    
    for terminal in duplicates:
        
        if terminal in df[locations[0]].values:
            ix=df[df[locations[0]]==terminal].index.tolist()
            for i in ix:     
                df.loc[i,locations[0]]=terminal+str("_")+df.loc[i,countries[0]]
        
        if terminal in df[locations[1]].values:
            ix=df[df[locations[1]]==terminal].index.tolist()
            for i in ix:     
                df.loc[i,locations[1]]=terminal+str("_")+df.loc[i,countries[1]]   
    return df

In [70]:
# check Null in dataset
df_miss=check_missData(df_journey)
print(df_miss.head())
print ("-----------------------------------------------------------------------------------------------------------")
print ("since there are too many Nulls, we will deal with null only after extracting data")
print ("-----------------------------------------------------------------------------------------------------------")

# a test case with few data is given here for a validation of function check_dup(). 
df_test=pd.read_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/journey_tests.txt",sep=",")
dups=check_dup(df_test,['De Kastri','Ulsan','caisd'])
print ("This is the test case for a dataframe with one duplicated terminal: 'De Kastri' ")
print ("The duplicated terminal in the checking terminals are: ",dups)
print ("Validated ! ")
print ("-----------------------------------------------------------------------------------------------------------")

# Find all the terminals
terminal_end=list(set(df_journey.journey_end_location_name))
terminal_start=list(set(df_journey.journey_start_location_name))
terminal_all=list(set(terminal_end+terminal_start))
terminal_all= [x for x in terminal_all if x == x] # Since nan is not equal to nan (nan != nan). nan==nan--> False
print ("The first ten terminals:",terminal_all[:10])
print ("The overall number of terminals from data are:",len(terminal_all))
print ("-----------------------------------------------------------------------------------------------------------")

terminal_all_1=list(set(df_ports.Terminal))

# check duplicated terminals inside jounery file using the function check_dup() !
duplicates_terms=check_dup(df_journey,terminal_all)
print ("Among all the terminals, the duplicate terminals are:\n",duplicates_terms)
print ("-----------------------------------------------------------------------------------------------------------")
print ("The number of duplicated terminals:",len(duplicates_terms))
print ("-----------------------------------------------------------------------------------------------------------")

                                 TotalNum  TotalPerc(\%)
journey_end_location_unlocode        6106       1.217958
journey_start_location_unlocode      5093       1.015896
journey_end_location_country         2336       0.465960
journey_end_location_type            1754       0.349869
journey_end_location_name            1754       0.349869
-----------------------------------------------------------------------------------------------------------
since there are too many Nulls, we will deal with null only after extracting data
-----------------------------------------------------------------------------------------------------------
This is the test case for a dataframe with one duplicated terminal: 'De Kastri' 
The duplicated terminal in the checking terminals are:  ['De Kastri']
Validated ! 
-----------------------------------------------------------------------------------------------------------
The first ten terminals: ['Limas Liquid Terminal', 'Setoda Harbour', 'San Juan', 'Karimu

<ipython-input-69-6c7af396b795>:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if counts_start !=[]:
<ipython-input-69-6c7af396b795>:77: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if counts_end !=[]:
<ipython-input-69-6c7af396b795>:77: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if counts_end !=[]:
<ipython-input-69-6c7af396b795>:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if counts_start !=[]:


Among all the terminals, the duplicate terminals are:
 ['Freeport', 'San Juan', 'Puerto Bolivar', 'Portland', 'Damen Ship Repair', 'Caldera', 'Tripoli', "St John's", 'Cartagena', 'Manzanillo', 'Louis Dreyfus Grain Terminal', 'San Lorenzo', 'San Pedro', 'Vancouver', 'Samsung', 'Matanzas', 'Georgetown']
-----------------------------------------------------------------------------------------------------------
The number of duplicated terminals: 17
-----------------------------------------------------------------------------------------------------------


### Extract other corresponding data information of the duplicate terminals from the original journey file.
Note that I manually collect the duplicate information based on found dupliate terminals. 

In [71]:
def output_dup(df,dups=['Tripoli', 'Vancouver']):
    """
    extract the relevant information of duplicate terminals from original journey file based on given terminals
    it is only suitable for dataframe with columns: journey_start_location_name, journey_start_location_country,
    journey_start_latitude and journey_start_longitude.
    use the starting points to trace other relevant information. 
    
    df: dataFrame
    dups: duplicate terminals
    return: dictionaries including the number of cuntries of each terminal, countries, latitudes, and longitudes.
    """
    countries={}
    lats={}
    lons={}
    country_num={}
    for terminal in dups:
        ix=df[df["journey_start_location_name"]==terminal].index.values
        for i in ix:
            countries.setdefault(terminal,[]).append(df.loc[i,"journey_start_location_country"])
            lats.setdefault(terminal,[]).append(df.loc[i,"journey_start_latitude"])
            lons.setdefault(terminal,[]).append(df.loc[i,"journey_start_longitude"])
        country_num.setdefault(terminal,[]).append(len(set(countries[terminal])))
    return country_num,countries,lats,lons

country_num,countries,lats,lons=output_dup(df_journey,['Samsung'])
print ("The number of countries the terminals belong to:\n",country_num)
print ("----------------------------------------------------------------------------------------------------------------")
print("The corresponding countries for each duplicate terminals:\n",countries)
print ("----------------------------------------------------------------------------------------------------------------")
print("The corresponding latitude for each duplicate terminals:\n",lats)
print ("----------------------------------------------------------------------------------------------------------------")
print("The corresponding longitude for each duplicate terminals:\n",lons)

The number of countries the terminals belong to:
 {'Samsung': [2]}
----------------------------------------------------------------------------------------------------------------
The corresponding countries for each duplicate terminals:
 {'Samsung': ['South Korea', 'South Korea', 'China', 'South Korea', 'China', 'China', 'China', 'China', 'China', 'South Korea', 'China', 'South Korea', 'China', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'China', 'South Korea', 'South Korea', 'China', 'China', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'China', 'South Korea', 'South Korea', 'South Korea', 'South Korea', 'China', 'China', 'China', 'China', 'South Korea', 'China', 'South Korea', 'China']}
----------------------------------------------------------------------------------------------------------------
The corresponding latitude for each duplicate terminals:
 {'Samsung': [34.909909999999996, 34.915

In [72]:
# find information for some exception duplicate terminals.
# The terminal Bermuda is hidding among a bunch of Bahamas. 
# Check the Bermuda port 
terminal_bermuda=df_journeyLoc[(df_journeyLoc.journey_end_location_country=="Bermuda") & (df_journeyLoc.journey_end_location_name=="Freeport")]
#print (terminal_bermuda.head())

# The terminal St John's is hidding among a bunch of Antigua & Barbuda. 
# Check the St John's port in Canada
terminal_st=df_journeyLoc[(df_journeyLoc.journey_end_location_country=="Canada") & (df_journeyLoc.journey_end_location_name=="St John's")]
#print (terminal_st.head())

# The terminal San Juan is hidding among a bunch of Puerto Rico. 
# Check the San Juan port in Panama
terminal_sj=df_journeyLoc[(df_journeyLoc.journey_end_location_country=="Puerto Rico") & (df_journeyLoc.journey_end_location_name=="San Juan")]
terminal_sj.head(2)


,journey_start_location_type,journey_start_location_country,journey_start_location_name,journey_start_latitude,journey_start_longitude,journey_end_location_type,journey_end_location_country,journey_end_location_name,journey_end_latitude,journey_end_longitude
5989,Port,Caribbean Netherlands,St Eustatius,17.50264,-63.00621,Port,Puerto Rico,San Juan,18.43056,-66.11004
6652,Port,Lithuania,Klaipeda Nafta Oil Terminal,55.72612,21.09402,Port,Puerto Rico,San Juan,18.43065,-66.10993


####  Load the found duplicate terminals informaiton, and update the duplicate terminals in original file

In [73]:
# read the extracted duplicate data, which are based on the cleaning above. 
df_dupTerm=pd.ExcelFile("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/portList_duplicate.xlsx")
df_dupTerm=df_dupTerm.parse(header=0,index_col=0)

# the duplicated terminals computed above are presented here as well
print ("Among all the terminals, the duplicate terminals are:\n",duplicates_terms)
print ("-------------------------------------------------------------------------------------------------------------------")

# update the name of duplicated terminals in original journey file.
df_journey_copy=df_journey.copy()
df_journey_new=updateName_dup(df_journey_copy,duplicates_terms)

# re-check the new dataframe if there are duplicates
duplicates_terms1=check_dup(df_journey_new,terminal_all)
print ("Duplicates in the dataset with updated names",duplicates_terms1)

# re-check the number of duplicates, and to validate the new dataset
terminals_1=list(set(df_journey_new.journey_start_location_name))
terminals_2=list(set(df_journey_new.journey_end_location_name))
terminals_new_all=list(set(terminals_1+terminals_2))
terminals_new_all=[x for x in terminals_new_all if x==x]
print ("---------------------------------------------------------------------------------------------------------------------")
print ("The new overall number of terminals are:",len(terminals_new_all))
print ("---------------------------------------------------------------------------------------------------------------------")
print("The first ten terminals:\n",terminals_new_all[:10])
print ("---------------------------------------------------------------------------------------------------------------------")
print("As we noticed, the added terminal name is 17 due to duplicates. Thus, the overall terminals should be 3288 in total \n, matching with the number we account.")
print ("New dataset is clean with no Duplicats !!!!!")
print ("Great ! No duplicates !!!")
print ("---------------------------------------------------------------------------------------------------------------------")

Among all the terminals, the duplicate terminals are:
 ['Freeport', 'San Juan', 'Puerto Bolivar', 'Portland', 'Damen Ship Repair', 'Caldera', 'Tripoli', "St John's", 'Cartagena', 'Manzanillo', 'Louis Dreyfus Grain Terminal', 'San Lorenzo', 'San Pedro', 'Vancouver', 'Samsung', 'Matanzas', 'Georgetown']
-------------------------------------------------------------------------------------------------------------------


<ipython-input-69-6c7af396b795>:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if counts_start !=[]:
<ipython-input-69-6c7af396b795>:77: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if counts_end !=[]:


Duplicates in the dataset with updated names []
---------------------------------------------------------------------------------------------------------------------
The new overall number of terminals are: 3288
---------------------------------------------------------------------------------------------------------------------
The first ten terminals:
 ['Limas Liquid Terminal', 'Setoda Harbour', 'Karimun Besar', 'Koje Marine Terminal', 'Brass Terminal', 'Richards Bay Dry Bulk Terminal', 'Keelung North Container Terminal', 'Jebel Ali Container Terminal 2', 'Vado Ligure', 'Fort de France']
---------------------------------------------------------------------------------------------------------------------
As we noticed, the added terminal name is 17 due to duplicates. Thus, the overall terminals should be 3288 in total 
, matching with the number we account.
New dataset is clean with no Duplicats !!!!!
Great ! No duplicates !!!
-----------------------------------------------------------

### Step d): Find the number of calls for new cleaned dataset

In [86]:

def check_valsCount(df):
    """
    check the number of counts of each value in dataframe
    df: a dataframe
    return: a list including all the counts of each column
    """
    cols=df.columns
    counts_all=[]
    for col in cols:
        counts=df.loc[:,col].value_counts()
        counts_all.append(counts)
        print (counts)
        print ('\n')
    return counts_all

def drop_dups(df,dups=['Cartagena', 'Freeport']):
    """
    Given a list of terminals, obtaining a new dataframe by removing these terminals.
    df:dataframe
    dups: list including removed terminals
    Note that the original dataframe is modified here. 
    lease make a copy of the orignal dataframe.
    
    return: a new dataframe
    """
    cols=["journey_start_location_name","journey_end_location_name"]
    for terminal in dups:
        # get the index of duplicated terminal
        ix_1=df[df["journey_start_location_name"]==terminal].index.tolist()
        ix_2=df[df["journey_end_location_name"]==terminal].index.tolist()
        ix=ix_1+ix_2
        
        df.drop(ix,axis=0,inplace=True)
    return df


def findCalls(df):
    """
    to compute the number of calls for each terminal based on the number of occurrence in both start port or end port. 
    save results into a dict, with each terminal as the key.
    
    df: dataframe
    return: a new dataframe with columns:"Terminal","number of call start","number of call end","number of call all".    
    """
    cols=["journey_start_location_name","journey_end_location_name"]
    counts_all=[]
    for col in cols:
        counts=df.loc[:,col].value_counts()
        counts_all.append(counts)
    df_counts=pd.concat(counts_all,axis=1)
    df_counts["number of call all"]=df_counts.sum(axis=1)
    df_counts=df_counts.reset_index()
    df_counts.columns=["Terminal","number of call start","number of call end","number of call all"]
    df_counts.fillna(0,inplace=True)
    
    return df_counts

def find_Country_Loction(df,terminals=[]):
    """
    find other information for a given list of terminals, including countries, and location info, etc.
    
    df:dataframe
    return: a dataframe including terminals and other information.   
    """
    countries={}
    lats={}
    lons={}
    
    if terminals is None:
        return 
    
    for terminal in terminals:
        if terminal in df.journey_start_location_name.values:
            ix=df[df["journey_start_location_name"]==terminal].index[0]
            countries.setdefault(terminal,[]).append(df.loc[ix,"journey_start_location_country"])
            lats.setdefault(terminal,[]).append(df.loc[ix,"journey_start_latitude"])
            lons.setdefault(terminal,[]).append(df.loc[ix,"journey_start_longitude"])
        elif terminal in df.journey_end_location_name.values:
            ix=df[df["journey_end_location_name"]==terminal].index[0]
            countries.setdefault(terminal,[]).append(df.loc[ix,"journey_end_location_country"])
            lats.setdefault(terminal,[]).append(df.loc[ix,"journey_end_latitude"])
            lons.setdefault(terminal,[]).append(df.loc[ix,"journey_end_longitude"])
    countries=pd.DataFrame(countries).T
    lat=pd.DataFrame(lats).T
    lon=pd.DataFrame(lons).T
    conLoc=pd.concat([countries,lat,lon],axis=1)
    conLoc=conLoc.reset_index()
    conLoc.columns=["Terminal","Country","Lat","Lon"]
    return conLoc   
    
def merge_df(df1,df2,byCol="Terminal",how="outer"):
    """
    merge given dataframes
    df1:dataframe1
    df2:dataframe2
    byCol: the index colomn for merge
    how: the way to merge
    
    return: a new merged dataframe
    """
    df_all=pd.merge(df1,df2,how=how,on=byCol)
    return df_all

def saveDf(df,dirs="C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile/",fileName="portsInfo.csv"):
    """
    save dataframe into csv file
    df: dataframe needed to be save
    
    dirs: directory to be saved, string
    fileName: string
    return: None
    """
    df.to_csv(dirs+fileName,index=False)

In [75]:
df_journey_new.head()

,journey_id,ship_id,ais_ship_id,journey_laden,journey_start_stoppage_id,journey_start_timestamp,journey_start_draft,journey_start_latitude,journey_start_longitude,journey_start_location_type,...,journey_end_location_name,journey_end_location_country,journey_end_location_unlocode,journey_distance_nm,journey_underway_calc_avg_speed,journey_processed,journey_current,journey_estimated_cargo_tons,journey_underway_distance,journey_underway_time_hours
0,286559091,9301421,97,1,886369008,2013-12-30 05:34:03,9.0,51.45277,140.92767,Port,...,Yeosu,South Korea,KRYOS,1247.352457,13.959216,2019-03-28 06:53:38,0,98235,1195.141506,85.616667
1,286559092,9301421,97,0,886369009,2014-01-07 13:30:04,14.2,34.71473,127.81125,Anchorage,...,De Kastri,Russia,RUDKA,1251.511628,13.074002,2019-03-28 06:53:38,0,0,1233.310535,94.333056
2,286559093,9301421,97,1,886369012,2014-01-19 01:02:19,9.0,51.48012,140.92478,Anchorage,...,Ulsan,South Korea,KRUSN,1166.283499,14.453006,2019-03-28 06:53:38,0,98235,1132.023627,78.324444
3,286559094,9301421,97,0,886369014,2014-01-25 05:06:44,14.2,35.44117,129.39453,Port,...,De Kastri,Russia,RUDKA,1339.352542,13.024184,2019-03-28 06:53:38,0,0,1197.562825,91.949167
4,286559095,9301421,97,1,886369018,2014-02-05 12:23:14,9.0,51.48157,140.92332,Port,...,Gwangyang,South Korea,KRKAN,1299.165114,13.671880,2019-03-28 06:53:38,0,98235,1288.684831,94.258056


In [76]:
# find all the number of calls for each terminal
df_counts_new=findCalls(df_journey_new)

# find other information related to terminals
terminals_new=df_counts_new.Terminal.values
df_countries_new=find_Country_Loction(df_journey_new,terminals_new)

# merge the two datafarmes
df_all=merge_df(df_countries_new,df_counts_new,byCol="Terminal",how="outer")

In [77]:
# Check the new data
df_counts_new.head()

,Terminal,number of call start,number of call end,number of call all
0,Singapore,7403.0,7365.0,14768.0
1,PTP Terminal,3785.0,3781.0,7566.0
2,Novorossiysk,3762.0,3762.0,7524.0
3,Yeosu,3531.0,3547.0,7078.0
4,Houston,3371.0,3375.0,6746.0


In [78]:
df_countries_new.head()

,Terminal,Country,Lat,Lon
0,Singapore,Singapore,1.20470,103.67232
1,PTP Terminal,Russia,60.32058,28.72898
2,Novorossiysk,Russia,44.72680,37.78154
3,Yeosu,South Korea,34.71473,127.81125
4,Houston,United States of America,29.23550,-94.62350


In [79]:
df_all.head()

,Terminal,Country,Lat,Lon,number of call start,number of call end,number of call all
0,Singapore,Singapore,1.20470,103.67232,7403.0,7365.0,14768.0
1,PTP Terminal,Russia,60.32058,28.72898,3785.0,3781.0,7566.0
2,Novorossiysk,Russia,44.72680,37.78154,3762.0,3762.0,7524.0
3,Yeosu,South Korea,34.71473,127.81125,3531.0,3547.0,7078.0
4,Houston,United States of America,29.23550,-94.62350,3371.0,3375.0,6746.0


### Final round of checking, and save.

In [81]:
# Final round of checking for the cleaned dataset df_all
check_missData(df_all)

# Fill the Country in NaN with "unknown"
df_all=fill_null(df_all,column="Country",value="unknown")

# to check if any ohter Null values
df_null=check_null(df_all,column="Country")
print ("---------------------------------------------------------------------------------------------------------------")
print (df_null.info())
print ("---------------------------------------------------------------------------------------------------------------")
print ("Success")

---------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Terminal              0 non-null      object 
 1   Country               0 non-null      object 
 2   Lat                   0 non-null      float64
 3   Lon                   0 non-null      float64
 4   number of call start  0 non-null      float64
 5   number of call end    0 non-null      float64
 6   number of call all    0 non-null      float64
dtypes: float64(5), object(2)
memory usage: 0.0+ bytes
None
---------------------------------------------------------------------------------------------------------------
Success


In [87]:
saveDf(df_all,dirs="C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile/",fileName="portsInfo.csv")
print ("The cleaned portInfor.csv has been sucessfully saved to the file 'C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile'")

The cleaned portInfor.csv has been sucessfully saved to the file 'C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile'


In [ ]:
from pandas.io.excel import ExcelWriter
import pandas

# save data from csv to excel format
csv_files = ['portsInfo_1.csv']
with ExcelWriter("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/"+'portsInfo.xlsx') as ew:
    for csv_file in csv_files:
        pandas.read_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/"+csv_file).to_excel(ew, sheet_name=csv_file)